<a href="https://colab.research.google.com/github/Rob-A-B/wildfire-hotspots/blob/main/queimadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
from google.colab import drive
import gc

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
hoje = datetime.date.today()
BASE   = "/content/projeto_pcb"
BRONZE_RAW = f"{BASE}/dados/bronze/raw"                   # onde guardamos os arquivos "brutos" por partição
BRONZE_CUR = f"{BASE}/dados/bronze/current"               # dataset consolidado (único arquivo)
os.makedirs(BRONZE_RAW, exist_ok=True)
os.makedirs(BRONZE_CUR, exist_ok=True)

# comentario teste de commit

# Define the path to your CSV files in Google Drive
# **IMPORTANT:** Replace 'My Drive/Your_Folder_With_CSVs' with the actual path to your folder
DRIVE_CSV_PATH = "/content/drive/My Drive/8_periodo/Big Data/datasets"
os.makedirs(DRIVE_CSV_PATH, exist_ok=True) # Create the directory if it doesn't exist

In [4]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
from google.colab import drive
import dask.dataframe as dd

# Mount Google Drive
drive.mount('/content/drive')

hoje = datetime.date.today()
BASE   = "/content/projeto_pcb"
BRONZE_RAW = f"{BASE}/dados/bronze/raw"                   # onde guardamos os arquivos "brutos" por partição
BRONZE_CUR = f"{BASE}/dados/bronze/current"               # dataset consolidado (único arquivo)
os.makedirs(BRONZE_RAW, exist_ok=True)
os.makedirs(BRONZE_CUR, exist_ok=True)

# Define the path to your CSV files in Google Drive
# **IMPORTANT:** Replace 'My Drive/Your_Folder_With_CSVs' with the actual path to your folder
DRIVE_CSV_PATH = "/content/drive/My Drive/8_periodo/Big Data/datasets" # Assuming this is the correct path from previous execution
os.makedirs(DRIVE_CSV_PATH, exist_ok=True) # Create the directory if it doesn't exist


# 1) Descobrir todos os Parquets diretamente em /content/drive/My Drive/Your_Folder_With_Parquets (Assuming you have converted CSVs to Parquet)
# NOTE: If your files are still CSV, you'll need to convert them to Parquet first.
arquivos_parquet = [f for f in os.listdir(DRIVE_CSV_PATH) if f.endswith(".parquet") and os.path.isfile(f"{DRIVE_CSV_PATH}/{f}")]
print(f"📦 Encontrados {len(arquivos_parquet)} Parquet em {DRIVE_CSV_PATH}.\n")

# 2) Mover/copy to Bronze/raw partitioned and mount list of DataFrames
dfs = []
for nome in arquivos_parquet:
    caminho_src = f"{DRIVE_CSV_PATH}/{nome}"

    # extrai AAAAMM do nome (ex.: focos_mensal_br_202311.parquet)
    m = re.search(r'(\d{6})', nome)
    aaaamm = m.group(1) if m else f"{hoje.year}{hoje:%m}"
    ano, mes = aaaamm[:4], aaaamm[4:]

    # destino particionado
    dest_dir = os.path.join(BRONZE_RAW, f"ano={ano}", f"mes={mes}")
    os.makedirs(dest_dir, exist_ok=True)
    caminho_dst = os.path.join(dest_dir, nome)

    # Copy (to keep the original in Drive)
    shutil.copy(caminho_src, caminho_dst)
    print(f"✅ Copiado: {nome}  ➜  {caminho_dst}")


# 3) Consolidate everything into a single Dask DataFrame (Bronze/current)
# Read all parquet files from the raw bronze directory into a single Dask DataFrame
bronze_ddf = dd.read_parquet(os.path.join(BRONZE_RAW, "*/*/*.parquet"))

# Simple idempotence: remove exact duplicate rows
# (If desired, change to subset=["id","datahora","latitude","longitude"] if they exist)
bronze_ddf = bronze_ddf.drop_duplicates()

print(f"\n🧱 Bronze consolidated: {len(bronze_ddf):,} linhas, {bronze_ddf.shape[1]} colunas")
# For Dask, we use .head() to see the first few rows. This triggers computation.
display(bronze_ddf.head())

# 4) Salva consolidado (Parquet)
# We will save the consolidated data as a single Parquet file for efficiency with Dask
out_parquet = os.path.join(BRONZE_CUR, "focos_bronze_consolidado.parquet")

# To save as a single file, we repartition to 1 partition before saving
bronze_ddf.repartition(npartitions=1).to_parquet(out_parquet)


print(f"\n💾 Consolidado salvo em:\n - {out_parquet}")

📦 Encontrados 34 CSVs em /content/drive/My Drive/8_periodo/Big Data/datasets.

✅ Copiado: focos_mensal_br_202504.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=04/focos_mensal_br_202504.csv
✅ Copiado: focos_mensal_br_202501.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=01/focos_mensal_br_202501.csv
✅ Copiado: focos_mensal_br_202502.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=02/focos_mensal_br_202502.csv
✅ Copiado: focos_mensal_br_202503.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=03/focos_mensal_br_202503.csv


KeyboardInterrupt: 

In [5]:
!pip install dask dask[dataframe] pyarrow

In [ ]:
bronze_df.head()

In [ ]:
bronze_df.info()

In [ ]:
bronze_df.isna().sum()

In [ ]:
hoje = datetime.date.today()
BASE = "/content/projeto_pcb"
BRONZE = "/content/projeto_pcb/dados/bronze/current"
SILVER = "/content/projeto_pcb/dados/silver"
os.makedirs(SILVER, exist_ok=True)

In [ ]:
df_temp = None
del df_temp
gc.collect()

In [ ]:
bronze_df.drop(columns=['id', 'lat','lon','municipio_id','estado_id','pais_id','pais','origem_arquivo', 'ano', 'mes'], inplace=True)

In [ ]:
import os, glob, datetime
import pandas as pd
import numpy as np


paths = sorted(glob.glob(os.path.join(BRONZE, "*.csv")))
assert paths, "Nenhum CSV encontrado em bronze."

#LEMBRAR dessa linha mais na frente quando tiver mais meses / tratar isso no bronze
#df = pd.concat((pd.read_csv(p) for p in paths), ignore_index=True)

esperadas = {"data_hora_gmt","satelite","municipio","estado"}
faltando = esperadas - set(bronze_df.columns)
assert not faltando, f"Faltam colunas no CSV: {faltando}"

bronze_df["data_hora_gmt"] = pd.to_datetime(bronze_df["data_hora_gmt"], errors="coerce", utc=True)

for c in ["satelite","municipio","estado"]:
    bronze_df[c] = bronze_df[c].astype(str).str.strip()


bronze_df = bronze_df.dropna(subset=["data_hora_gmt"]).copy()

# Identify and replace -999 with NaN in numeric columns
numeric_cols = bronze_df.select_dtypes(include=np.number).columns
bronze_df[numeric_cols] = bronze_df[numeric_cols].replace(-999, np.nan)

# Calculate and print proportion of NaNs per column
nan_proportion_per_column = bronze_df.isnull().sum() / len(bronze_df) * 100
print("\nProporção de NaN por coluna (%):")
print(nan_proportion_per_column)

# Calculate and print proportion of rows with NaN
rows_with_nan = bronze_df.isnull().any(axis=1).sum()
proportion_rows_with_nan = rows_with_nan / len(bronze_df) * 100
print(f"\nProporção de linhas com NaN no total do dataset (%): {proportion_rows_with_nan:.2f}%")


# Drop rows with NaN values
bronze_df = bronze_df.dropna().copy()


bronze_df["ano"] = bronze_df["data_hora_gmt"].dt.year.astype("Int64")
bronze_df["mes"] = bronze_df["data_hora_gmt"].dt.month.astype("Int64")
bronze_df["dia"] = bronze_df["data_hora_gmt"].dt.day.astype("Int64")

# carga que prof pediu
bronze_df["dt_carga"] = datetime.date.today().isoformat()

# Salva silver
silver_csv = os.path.join(SILVER, f"focos_silver_{datetime.date.today():%Y%m%d}.csv")
os.makedirs(SILVER, exist_ok=True)
bronze_df.to_csv(silver_csv, index=False)

print("Silver salvo:", silver_csv, "| linhas:", len(bronze_df))
bronze_df.head()

In [19]:
bronze_df.isna().sum()

,0
data_hora_gmt,0
satelite,0
municipio,0
estado,0
numero_dias_sem_chuva,302472
precipitacao,302472
risco_fogo,302472
bioma,9
frp,511105
ano,0


In [ ]:
bronze()

In [ ]:
GOLD = "/content/projeto_pcb/dados/gold
os.makedirs(GOLD, exist_ok=True)

In [ ]:
padrao_silver = "focos_silver_*.csv"   # ajuste se usou outro nome

paths = sorted(glob.glob(os.path.join(SILVER, padrao_silver)))
assert paths, f"Nenhum CSV Silver encontrado em: {SILVER}"

# parse_dates  reforçandos a conversão em seguida
df = pd.concat((pd.read_csv(p, parse_dates=["data_hora_gmt"]) for p in paths),
               ignore_index=True)

# Garante que data_hora_gmt é datetime
df["data_hora_gmt"] = pd.to_datetime(df["data_hora_gmt"], errors="coerce", utc=True)

# Normalizações úteis para agregação
df["data"] = df["data_hora_gmt"].dt.date
df["ano"]  = df["data_hora_gmt"].dt.year
df["mes"]  = df["data_hora_gmt"].dt.month
df["hora"] = df["data_hora_gmt"].dt.hour

por_estado_mes = (
    df.groupby(["estado","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)


por_municipio_dia = (
    df.groupby(["estado","municipio","data"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["data","qtd_focos"], ascending=[True,False])
)


por_satelite_mes = (
    df.groupby(["satelite","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)

por_satelite_total = (
    df.groupby("satelite")
      .size()
      .reset_index(name="qtd_focos")
      .assign(perc=lambda x: (x["qtd_focos"] / x["qtd_focos"].sum())*100)
      .sort_values("qtd_focos", ascending=False)
      .round({"perc": 2})
)

df["lat_cell"] = df["lat"].round(2)
df["lon_cell"] = df["lon"].round(2)
grade_espacial = (
    df.groupby(["lat_cell","lon_cell"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values("qtd_focos", ascending=False)
)

# ------------------------------
# Salva em GOLD
# ------------------------------
os.makedirs(GOLD, exist_ok=True)
hoje = datetime.date.today().isoformat()

g1 = os.path.join(GOLD, f"gold_focos_por_estado_mes_{hoje}.csv")
g2 = os.path.join(GOLD, f"gold_focos_por_municipio_dia_{hoje}.csv")
g3 = os.path.join(GOLD, f"gold_focos_por_satelite_mes_{hoje}.csv")
g3b= os.path.join(GOLD, f"gold_focos_por_satelite_total_{hoje}.csv")
g4 = os.path.join(GOLD, f"gold_grade_espacial_{hoje}.csv")

por_estado_mes.to_csv(g1, index=False)
por_municipio_dia.to_csv(g2, index=False)
por_satelite_mes.to_csv(g3, index=False)
por_satelite_total.to_csv(g3b, index=False)
grade_espacial.to_csv(g4, index=False)

print("Gold salvo:")
print(" -", g1)
print(" -", g2)
print(" -", g3)
print(" -", g3b)
print(" -", g4)

try:
    ult_ano  = por_estado_mes["ano"].max()
    ult_mes  = por_estado_mes.query("ano == @ult_ano")["mes"].max()
    top_estados = (
        por_estado_mes
        .query("ano == @ult_ano and mes == @ult_mes")
        .nlargest(10, "qtd_focos")
    )
    print(f"\nTOP estados no mês mais recente ({ult_ano}-{ult_mes:02d}):")
    print(top_estados.to_string(index=False))
except Exception as e:
    print("\n[Aviso] Não foi possível imprimir TOP estados do mês mais recente:", e)

print("\n% por satélite (total):")
print(por_satelite_total.to_string(index=False))

Gold salvo:
 - /content/projeto_pcb/dados/gold/gold_focos_por_estado_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_municipio_dia_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_total_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_grade_espacial_2025-10-09.csv

TOP estados no mês mais recente (2024-03):
            estado  ano  mes  qtd_focos
           RORAIMA 2024    3       1767
       MATO GROSSO 2024    3       1508
             BAHIA 2024    3        936
MATO GROSSO DO SUL 2024    3        562
             GOIÁS 2024    3        429
         SÃO PAULO 2024    3        282
      MINAS GERAIS 2024    3        260
         TOCANTINS 2024    3        132
              PARÁ 2024    3         98
           ALAGOAS 2024    3         93

% por satélite (total):
 satelite  qtd_focos  perc
  NOAA-20       1870 28.18
  NPP-375       1399 21.08
 NPP-375D      

In [ ]:
# cont = df["Grey"].value_counts().sort_index()
# plt.bar(cont.index.astype(str), cont.values)
# plt.title("Distribuição da classe Grey")
# plt.xlabel("Grey (0=não cinza, 1=cinza)")
# # plt.ylabel("n")
# plt.show()


In [ ]:
# agg = df.groupby("Grey")[["R","G","B"]].mean().reset_index()
# for ch in ["R","G","B"]:
#     plt.bar(agg["Grey"].astype(str), agg[ch], label=ch)
#     plt.title(f"Média do canal {ch} por Grey")
#     plt.xlabel("Grey")
#     plt.ylabel(f"Média {ch} (escala {'0–255' if df['R'].max()>1 else '0–1'})")
#     plt.show()
